L'objectif de la task 2 sera de récupérer les parquets individuels crée, de les assembler et de les envoyer dans un méga parquet

In [1]:
import gzip 
from lxml import etree
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 
import pyarrow.parquet as pq


def timtim(fonction):
    def wrapper(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        temps_execution = fin - debut
        print(f"La fonction {fonction.__name__} a pris {temps_execution:.5f} secondes pour s'exécuter.")
        return resultat
    return wrapper

In [2]:
DOSSIER_PARQUETS_INDIVIDUELS = './fichiers/todo_parquet/'

In [3]:
liste_parquet_t = []
for parquet in glob.glob(os.path.join(DOSSIER_PARQUETS_INDIVIDUELS, "*")) : 
 liste_parquet_t.append(parquet)

In [ ]:
liste_parquet_t 

['./fichiers/todo_parquet\\612019_pyarrow',
 './fichiers/todo_parquet\\612023_pyarrow',
 './fichiers/todo_parquet\\612027_pyarrow',
 './fichiers/todo_parquet\\612030_pyarrow',
 './fichiers/todo_parquet\\612038_pyarrow',
 './fichiers/todo_parquet\\612050_pyarrow',
 './fichiers/todo_parquet\\612057_pyarrow',
 './fichiers/todo_parquet\\612058_pyarrow',
 './fichiers/todo_parquet\\612069_pyarrow',
 './fichiers/todo_parquet\\612078_pyarrow',
 './fichiers/todo_parquet\\612117_pyarrow',
 './fichiers/todo_parquet\\612127_pyarrow',
 './fichiers/todo_parquet\\612128_pyarrow',
 './fichiers/todo_parquet\\612131_pyarrow',
 './fichiers/todo_parquet\\612136_pyarrow',
 './fichiers/todo_parquet\\612151_pyarrow',
 './fichiers/todo_parquet\\612159_pyarrow',
 './fichiers/todo_parquet\\612160_pyarrow',
 './fichiers/todo_parquet\\612172_pyarrow',
 './fichiers/todo_parquet\\612180_pyarrow',
 './fichiers/todo_parquet\\612188_pyarrow',
 './fichiers/todo_parquet\\612192_pyarrow',
 './fichiers/todo_parquet\\612220_pyarrow',
 './fichiers/todo_parquet\\612221_pyarrow',
 './fichiers/todo_parquet\\612223_pyarrow',
...

In [5]:
schema = pq.ParquetFile(liste_parquet_t[0]).schema_arrow
with pq.ParquetWriter("grosparquet.parquet", schema=schema) as writer:
    for file in liste_parquet_t:
        table = pq.read_table(file, schema=schema)
        writer.write_table(table)


In [6]:
fichier_concatene = "grosparquet.parquet"
table_polars = pl.read_parquet(fichier_concatene)
print(table_polars)

shape: (59_875, 23)
┌────────┬─────────────┬─────────┬────────┬───┬─────────────┬────────────┬────────────┬────────────┐
│ Nature ┆ LibCpte     ┆ ContNat ┆ ArtSpe ┆ … ┆ IdColl      ┆ NatCEPL    ┆ Departemen ┆ Id_Fichier │
│ ---    ┆ ---         ┆ ---     ┆ ---    ┆   ┆ ---         ┆ ---        ┆ t          ┆ ---        │
│ str    ┆ str         ┆ str     ┆ str    ┆   ┆ str         ┆ str        ┆ ---        ┆ str        │
│        ┆             ┆         ┆        ┆   ┆             ┆            ┆ str        ┆            │
╞════════╪═════════════╪═════════╪════════╪═══╪═════════════╪════════════╪════════════╪════════════╡
│ 60623  ┆ Alimentatio ┆ 011     ┆ false  ┆ … ┆ 26780487000 ┆ Commune    ┆ 78         ┆ 612019     │
│        ┆ n pour les  ┆         ┆        ┆   ┆ 017         ┆            ┆            ┆            │
│        ┆ cantines e… ┆         ┆        ┆   ┆             ┆            ┆            ┆            │
│ 60628  ┆ Autres      ┆ 011     ┆ false  ┆ … ┆ 26780487000 ┆ Commune  

Gros parquet n'est pas stocké au bon endroit, mais c'est pas un vrai soucis, pour le moment. on va juste comparer avec pandas pour la vitesse d'execution

In [7]:
table_pandas = pd.read_parquet(fichier_concatene)
table_pandas.head(5)

,Nature,LibCpte,ContNat,ArtSpe,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,...,MtSup,CaracSup,DteStr,LibellePoste,IdPost,LibelleColl,IdColl,NatCEPL,Departement,Id_Fichier
0,60623,Alimentation pour les cantines et les colonies...,011,false,D,6000,0,0,0,3000,...,"{'@Code': 'BudgetHorsRAR', '@V': '3000'}","{'@Code': 'ProgAutoNum', '@V': '0'}",2019-01-01,CENTRE DES FINANCES PUBLIQUES,078027,Caisse des Ecoles de Bullion,26780487000017,Commune,78,612019
1,60628,Autres fournitures non stockées,011,false,D,750,0,0,0,350,...,"{'@Code': 'BudgetHorsRAR', '@V': '350'}","{'@Code': 'ProgAutoNum', '@V': '0'}",2019-01-01,CENTRE DES FINANCES PUBLIQUES,078027,Caisse des Ecoles de Bullion,26780487000017,Commune,78,612019
2,6063,"{'@V': ""Fournitures d'entretien et de petit éq...",011,false,D,16580,0,0,0,5950,...,"{'@Code': 'BudgetHorsRAR', '@V': '5950'}","{'@Code': 'ProgAutoNum', '@V': '0'}",2019-01-01,CENTRE DES FINANCES PUBLIQUES,078027,Caisse des Ecoles de Bullion,26780487000017,Commune,78,612019
3,6065,"Livres, disques, cassettes... (bibliothèques e...",011,false,D,6050,0,0,0,3800,...,"{'@Code': 'BudgetHorsRAR', '@V': '3800'}","{'@Code': 'ProgAutoNum', '@V': '0'}",2019-01-01,CENTRE DES FINANCES PUBLIQUES,078027,Caisse des Ecoles de Bullion,26780487000017,Commune,78,612019
4,6067,Fournitures scolaires,011,false,D,17774,0,0,0,8500,...,"{'@Code': 'BudgetHorsRAR', '@V': '8500'}","{'@Code': 'ProgAutoNum', '@V': '0'}",2019-01-01,CENTRE DES FINANCES PUBLIQUES,078027,Caisse des Ecoles de Bullion,26780487000017,Commune,78,612019


In [8]:
print(table_pandas.shape)
print(table_polars.shape)

(59875, 23)
(59875, 23)


On va maintenant comparer les deux sur 20 000 fichiers 

Note : c'est bordélique, faudra mettre ça au propre (je me base sur le même dossier pour un test à 500 et à 20 000 fichiers)

In [12]:
liste_parquet_20k = []
for parquet_20 in glob.glob(os.path.join(DOSSIER_PARQUETS_INDIVIDUELS, "*")) : 
 liste_parquet_20k.append(parquet_20)

schema20 = pq.ParquetFile(liste_parquet_20k[0]).schema_arrow
with pq.ParquetWriter("parquet20k.parquet", schema=schema20) as writer20:
    for file20 in liste_parquet_20k:
        table20 = pq.read_table(file20, schema=schema20)
        writer20.write_table(table20)

In [13]:
parquet20k = 'parquet20k.parquet'
table20_polars = pl.read_parquet(parquet20k)
table20_polars.shape

(2683082, 24)

In [14]:
table20_pandas = pd.read_parquet(parquet20k)
table20_pandas.shape

(2683082, 24)

Resultats sur 20k : 

Concatenage : 3min 3sec

Ouverture polars : 1.5sec

Ouverture pandas : 9.0sec 

In [15]:
table20_pandas.head(5)

,Nature,ContNat,ArtSpe,CodRD,MtBudgPrec,MtPropNouv,MtPrev,CredOuv,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Departement,Id_Fichier,TypOpBudg,MtRARPrec,LibCpte,Operation,ContOp
0,60632,011,false,D,11962.00,19335.00,19335.00,19335.00,0,"{'@V': '11962.00', '@Code': 'BudgetHorsRAR'}",...,AGONAC,21240002200019,Commune,24,612013,nan,nan,nan,nan,nan
1,6332,012,false,D,450.00,500.00,500.00,500.00,0,"{'@V': '450.00', '@Code': 'BudgetHorsRAR'}",...,AGONAC,21240002200019,Commune,24,612013,nan,nan,nan,nan,nan
2,6411,012,false,D,365000.00,352000.00,352000.00,352000.00,0,"{'@V': '365000.00', '@Code': 'BudgetHorsRAR'}",...,AGONAC,21240002200019,Commune,24,612013,nan,nan,nan,nan,nan
3,74834,nan,false,R,4104.00,4084.00,4084.00,4084.00,0,"{'@V': '4104.00', '@Code': 'BudgetHorsRAR'}",...,AGONAC,21240002200019,Commune,24,612013,nan,nan,nan,nan,nan
4,6331,012,false,D,4250.00,5200.00,5200.00,5200.00,0,"{'@V': '4250.00', '@Code': 'BudgetHorsRAR'}",...,AGONAC,21240002200019,Commune,24,612013,nan,nan,nan,nan,nan


In [16]:
table20_polars.head(5)

Nature,ContNat,ArtSpe,CodRD,MtBudgPrec,MtPropNouv,MtPrev,CredOuv,OpBudg,MtSup,CaracSup,DteStr,LibellePoste,IdPost,LibelleColl,IdColl,NatCEPL,Departement,Id_Fichier,TypOpBudg,MtRARPrec,LibCpte,Operation,ContOp
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""60632""","""011""","""false""","""D""","""11962.00""","""19335.00""","""19335.00""","""19335.00""","""0""","""{'@V': '11962.…","""{'@V': 'F', '@…","""2020-08-01""","""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""","""612013""","""nan""","""nan""","""nan""","""nan""","""nan"""
"""6332""","""012""","""false""","""D""","""450.00""","""500.00""","""500.00""","""500.00""","""0""","""{'@V': '450.00…","""{'@V': 'F', '@…","""2020-08-01""","""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""","""612013""","""nan""","""nan""","""nan""","""nan""","""nan"""
"""6411""","""012""","""false""","""D""","""365000.00""","""352000.00""","""352000.00""","""352000.00""","""0""","""{'@V': '365000…","""{'@V': 'F', '@…","""2020-08-01""","""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""","""612013""","""nan""","""nan""","""nan""","""nan""","""nan"""
"""74834""","""nan""","""false""","""R""","""4104.00""","""4084.00""","""4084.00""","""4084.00""","""0""","""{'@V': '4104.0…","""{'@V': 'F', '@…","""2020-08-01""","""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""","""612013""","""nan""","""nan""","""nan""","""nan""","""nan"""
"""6331""","""012""","""false""","""D""","""4250.00""","""5200.00""","""5200.00""","""5200.00""","""0""","""{'@V': '4250.0…","""{'@V': 'F', '@…","""2020-08-01""","""TRESORERIE DE …","""024003""","""AGONAC""","""21240002200019…","""Commune""","""24""","""612013""","""nan""","""nan""","""nan""","""nan""","""nan"""


Evidemment, si j'avais un schema propre ce serait mieux. 

Et voici un schema, vraiment pas propre : 


In [ ]:
custom_schema = pa.schema([
        ('Nature', pa.string()),
        ('Fonction', pa.string()),
        ('LibCpte', pa.string()),
        ('ContNat', pa.string()),
        ('ArtSpe', pa.string()), #Doit être corrigé en bool,
        ('ContFon', pa.string()),
        ('CodRD', pa.string()), #Val D ou R,
        ('MtBudgPrec', pa.string()), #float32 #nullable = True
        ('MtRARPrec', pa.string()), #float32
        ('MtPropNouv', pa.string()), #float32
        ('MtPrev', pa.string()), #float32
        ('CredOuv', pa.string()), #int32
        ('MtReal', pa.string()), #float32
        ('MtRAR3112', pa.string()), #float32
        ('OpBudg', pa.string()),
        ('MtSup', pa.string()),
        ('MtSup_1_Lib', pa.string()),
        ('MtSup_1_Val', pa.string()),
        ('MtSup_2_Lib', pa.string()),
        ('MtSup_2_Val', pa.string()),
        ('MtSup_3_Lib', pa.string()),
        ('MtSup_3_Val', pa.string()),
        ('MtSup_4_Lib', pa.string()),
        ('MtSup_4_Val', pa.string()),
        ('MtSup_5_Lib', pa.string()),
        ('MtSup_5_Val', pa.string()),
        ('MtSup_6_Lib', pa.string()),
        ('MtSup_6_Val', pa.string()),
        ('MtSup_7_Lib', pa.string()),
        ('MtSup_7_Val', pa.string()),
        ('MtSup_8_Lib', pa.string()),
        ('MtSup_8_Val', pa.string()),
        ('MtSup_9_Lib', pa.string()),
        ('MtSup_9_Val', pa.string()),
        ('MtSup_10_Lib', pa.string()),
        ('MtSup_10_Val', pa.string()), #10
        ('CaracSup_1_Lib', pa.string()),
        ('CaracSup_1_Val', pa.string()),
        ('CaracSup_2_Lib', pa.string()),
        ('CaracSup_2_Val', pa.string()),
        ('CaracSup_3_Lib', pa.string()),
        ('CaracSup_3_Val', pa.string()),
        ('CaracSup_4_Lib', pa.string()),
        ('CaracSup_4_Val', pa.string()),
        ('CaracSup_5_Lib', pa.string()),
        ('CaracSup_5_Val', pa.string()),
        ('CaracSup_6_Lib', pa.string()),
        ('CaracSup_6_Val', pa.string()),
        ('CaracSup_7_Lib', pa.string()), #7
        ('CaracSup_7_Val', pa.string()),
        ('TypOpBudg', pa.string()), #des 2 et des 1
        ('OpeCpteTiers', pa.string()),
        ('DteStr', pa.date32()),
        ('LibellePoste', pa.string()),
        ('IdPost', pa.string()),
        ('LibelleColl', pa.string()),
        ('IdColl', pa.string()),
        ('NatCEPL', pa.string()),
        ('Departement', pa.string()), #On oublie pas les 2A,
        ('Id_Fichier', pa.int64())
    ])

In [ ]:
def task_2_1(dossier, nom_parquet, schema_cust) :
 liste_parquet_t = []
 for parquet in glob.glob(os.path.join(dossier, "*")) : 
  liste_parquet_t.append(parquet)

 schema = schema_cust
 #schema = pq.ParquetFile(liste_parquet_t[0]).schema_arrow
 with pq.ParquetWriter(f"{nom_parquet}.parquet", schema=schema) as writer:
    for file in liste_parquet_t:
        table = pq.read_table(file, schema=schema)
        writer.write_table(table)